In [1]:
import pandas as pd
import time
from selenium.webdriver import Chrome
import re
import random
import time
from selenium.webdriver.chrome.options import Options
from textblob import TextBlob
import csv

timeDelay = random.randrange(6, 12)

In [2]:
def find_hashtags(title):
    hashtags = re.findall('#[A-Za-z]+', title)
    return hashtags

In [3]:
def recent_posts(tag):
    """With the input of an account page, scrape the 25 most recent posts urls"""
    url = "#" + tag +"/"
    browser.get(url)
    post = 'https://www.instagram.com/p/'
    post_links = []
    while len(post_links) < 100:
        links = [a.get_attribute('href') for a in browser.find_elements_by_tag_name('a')]
        for link in links:
            if post in link and link not in post_links:
                post_links.append(link)
        scroll_down = "window.scrollTo(0, document.body.scrollHeight);"
        browser.execute_script(scroll_down)
        time.sleep(timeDelay)
    else:
        return post_links[:100]

In [4]:
def post_details(url):
    """Take a post url and return post details"""
    post_details = None
    print(url)
    browser.get(url)

    try:
        xpath_title = '//*[@id="react-root"]/section/main/div/div/article/div[2]/div[1]/ul/div/li/div/div/div[2]/span'
        title = browser.find_element_by_xpath(xpath_title).text
    except Exception as e:
        print(e)
        title=None
    text = TextBlob(title).detect_language()
    
    if(text == 'en'):

        try:
            # This captures the standard like/views count. 
            xpath_likes = '//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/div/button/span'
            likes = browser.find_element_by_xpath(xpath_likes)
            if likes:
                likes=likes.text
            else:
                xpath_views = '//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/span'
                likes = browser.find_element_by_xpath(xpath_views)
                if likes:
                    likes=likes.text
                else:
                    likes=None

        except Exception as e:
            print(e)
            likes = None
        try:   
            age = browser.find_element_by_css_selector('a time').text
        except Exception as e:
            print(e)
            age=None

#         try:
#             xpath_comment = '//*[@id="react-root"]/section/main/div/div/article/div[2]/div[1]/ul/ul[1]/div/li/div/div/div[2]/span'
#             comment = browser.find_element_by_xpath(xpath_comment).text
#         except Exception as e:
#             print(e)
#             comment=None

        try:
            xpath_location = '//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a'
            location = browser.find_element_by_xpath(xpath_location).text
        except Exception as e:
            print(e)
            location=None

        try:    
            insta_link = url#.replace('https://www.instagram.com/p','')
        except Exception as e:
            print(e)
            insta_link=None

        try:
            xpath_username = '//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[1]/div[1]/h2/a'
            username = browser.find_element_by_xpath(xpath_username).text
        except Exception as e:
            print(e)
            username=None

        post_details = {'link': insta_link, 'username': username, 'likes/views': likes,'age': age, 'title': title, 'location': location}
    else:
        pass
    time.sleep(timeDelay)
    return post_details

In [5]:
def post_comments(url):
    print(url)
    browser.get(url)
    time.sleep(timeDelay)
    comments_list, users_list = [], []
    
    # approximately 12 comments per page
    click_count=0

    while(1):
        try:
            load_more = browser.find_element_by_class_name("glyphsSpriteCircle_add__outline__24__grey_9.u-__7")
            print(load_more)
            load_more.click()
            click_count+=1
            time.sleep(timeDelay)
        except Exception as e:
            print(e)
            if click_count == 0:
                click_count+=1
                continue
            else:
                break

    print("final click count: " + str(click_count) + "; should yield roughly " + str(click_count*12) + " comments")
    comments = browser.find_elements_by_class_name("Mr508")

    for c in comments:
        comment = c.find_element_by_css_selector('span').get_attribute("textContent")
        user = c.find_element_by_class_name("TlrDj").get_attribute("textContent")
        #print("" + user + ": " + str(comment))
        comments_list.append(comment)
        users_list.append(user)
        
    print(len(comments_list))    
    return comments_list, users_list

In [6]:
browser = Chrome()
insta_posts = recent_posts("petfood") #pedigreedogfood,
posts_dataframe = pd.DataFrame({"post_url": insta_posts})
posts_dataframe.to_csv("instagram_post_url_01-16.csv", index=False)

try:
    with open('instagram_post_url_01-16.csv', 'r') as f:
        posts = csv.reader(f)
        for post in list(posts)[1:]:
            insta_post_details = post_details(post[0])
            if insta_post_details:
                detail_dataframe = pd.DataFrame([insta_post_details])
                detail_dataframe['hashtags'] = detail_dataframe['title'].apply(lambda x: find_hashtags(x))
                detail_dataframe.to_csv("instagram_posts_petfood_01-16.csv", mode='a', index=False, header=False)
            else:
                pass
except Exception as e:
    print(e)
    
try:
    with open('instagram_posts_petfood_01-16.csv', 'r') as f:
        posts = csv.reader(f)
        for post in list(posts)[1:]:
            print('link: ', post[0])
            comments, users = post_comments(post[0])
            link = [post[0] for x in comments]
            comment_dataFrame = pd.DataFrame({"link": link, "user": users, "comment": comments})
            comment_dataFrame.to_csv("instagram_comments_01-16.csv", mode='a', index=False, header=False)
except Exception as e:
    print(e)

# for post in posts:
#     comments, users = post_comments(post)
#     link = [post.replace('https://www.instagram.com/p','') for x in comments]
#     comment_dataFrame = pd.DataFrame({"link": link, "user": users, "comment": comments})
#     #comment_dataFrame = dataFrame.append(data)
#     comment_dataFrame.to_csv("instagram_comments.csv", mode='a', index=False, header=False)

browser.quit()

https://www.instagram.com/p/B7mQJ_WnhVW/
https://www.instagram.com/p/B7IhGFTniiI/
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a"}
  (Session info: chrome=79.0.3945.130)

https://www.instagram.com/p/B7kmpg3HOe9/
https://www.instagram.com/p/B7px2hjCc1N/
https://www.instagram.com/p/B7DtGZOnFnA/
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a"}
  (Session info: chrome=79.0.3945.130)

https://www.instagram.com/p/B7N_I0Entlc/
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/div/article/div[2]/section[2]/div/div/button/span"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/div/articl

https://www.instagram.com/p/B7ry-cvn5Ib/
https://www.instagram.com/p/B7ryBwoAxT1/
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a"}
  (Session info: chrome=79.0.3945.130)

https://www.instagram.com/p/B7rx4GIniM9/
https://www.instagram.com/p/B7rw-CCAd5y/
https://www.instagram.com/p/B7rw3bZBl9U/
https://www.instagram.com/p/B7rvy1IhBma/
https://www.instagram.com/p/B7rviL7BqVg/
https://www.instagram.com/p/B7rvBclJ9_L/
https://www.instagram.com/p/B7ru1ekHW3d/
https://www.instagram.com/p/B7ruWz3IOSD/
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/div/article/header/div[2]/div[2]/div[2]/a"}
  (Session info: chrome=79.0.3945.130)

https://www.instagram.com/p/B7ruVLnibFC/
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="react-root"]/section/main/div/div/article/div[2]

37
link:  https://www.instagram.com/p/B7YLEEGpVlm/
https://www.instagram.com/p/B7YLEEGpVlm/
Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
4
link:  https://www.instagram.com/p/B7YKv5_HbEz/
https://www.instagram.com/p/B7YKv5_HbEz/
Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
1
link:  https://www.instagram.com/p/B7X7c-WIi-B/
https://www.instagram.com/p/B7X7c-WIi-B/
Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
1
link:  https://www.instagram.com/p/B7X6icyIHtm/
https://www.instagram

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
5
link:  https://www.instagram.com/p/B7XuXnAntr-/
https://www.instagram.com/p/B7XuXnAntr-/
Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
2
link:  https://www.instagram.com/p/B7Xt-WlDdfu/
https://www.instagram

...

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
0
link:  https://www.instagram.com/p/B7aR5EShAFK/
https://www.instagram.com/p/B7aR5EShAFK/
Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
0
link:  https://www.instagram.com/p/B7aPK_cB1Ze/
https://www.instagram

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
0
link:  https://www.instagram.com/p/B7aAAl6BBBK/
https://www.instagram.com/p/B7aAAl6BBBK/
Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
0
link:  https://www.instagram.com/p/B7Z-ehiCrxx/
https://www.instagram

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
0
link:  https://www.instagram.com/p/B7sLcBinujX/
https://www.instagram.com/p/B7sLcBinujX/
Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
0
link:  https://www.instagram.com/p/B7sI1vPFAhv/
https://www.instagram

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
0
link:  https://www.instagram.com/p/B7r7AZvptS_/
https://www.instagram.com/p/B7r7AZvptS_/
Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
1
link:  https://www.instagram.com/p/B7r5Ut9gvB4/
https://www.instagram

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
0
link:  https://www.instagram.com/p/B7rcwKzj0dg/
https://www.instagram.com/p/B7rcwKzj0dg/
Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

Message: no such element: Unable to locate element: {"method":"css selector","selector":".glyphsSpriteCircle_add__outline__24__grey_9.u-__7"}
  (Session info: chrome=79.0.3945.130)

final click count: 1; should yield roughly 12 comments
1
link:  https://www.instagram.com/p/B7rmeaZlsIq/
https://www.instagram